In [ ]:
from os import path

# Third-party
from astropy.table import Table
from astropy.table import vstack
import astropy.coordinates as coord
import astropy.units as u
from astropy.constants import G, c
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('apw-notebook')
%matplotlib inline
from scipy.stats import scoreatpercentile

from comoving_rv.log import logger
from comoving_rv.db import Session, Base, db_connect
from comoving_rv.db.model import (Run, Observation, TGASSource, SimbadInfo, PriorRV,
                                  SpectralLineInfo, SpectralLineMeasurement, RVMeasurement)

import matplotlib as mpl
mpl.rc('text', usetex=True)
mpl.rcParams['text.latex.preamble'] = [r"\usepackage{amsmath}"]

In [ ]:
# base_path = '/Volumes/ProjectData/gaia-comoving-followup/'
# db_path = path.join(base_path, 'db.sqlite')
# engine = db_connect(db_path)
# session = Session()

In [ ]:
tbl = Table.read('group_llr_dv_tbl.ecsv', format='ascii.ecsv')
rave_tbl = Table.read('RAVE_group_llr_dv_tbl.ecsv', format='ascii.ecsv')
all_tbl = vstack((tbl, rave_tbl))

In [ ]:
(rave_tbl['d_min'] < 200).sum(), ((rave_tbl['R_RV'] > rave_tbl['R_mu']) & (rave_tbl['d_min'] < 200)).sum()

In [ ]:
shaya = Table.read('../../data/shaya_olling2011.fit')

In [ ]:
dllr = tbl['R_RV'] - tbl['R_mu']
print('{0} are comoving'.format((dllr>0).sum()))

In [ ]:
np.random.seed(1234)

fig,ax = plt.subplots(1, 1, figsize=(5, 5))

ax.scatter(dllr, tbl['dv_50'], marker='.', alpha=0.7, linewidth=0)

ax.set_xlim(-30, 10)
ax.set_xlabel(r'$\mathcal{R}_{\rm RV} - \mathcal{R}_\mu$')
ax.set_ylabel(r'$|\boldsymbol{v}_1 - \boldsymbol{v}_2|$ ' + 
              '[{0}]'.format((u.km/u.s).to_string('latex_inline')))
ax.set_ylim(-2, 65)

# for a few random pairs, show error bars
bin_edges = np.arange(-30, 10+1, 5)
idxs = []
for l,r in zip(bin_edges[:-1], bin_edges[1:]):
    idx, = np.where((dllr > l) & (dllr < r))
    idx = np.random.choice(idx, size=1, replace=False)
    idxs.append(idx)
idxs = np.ravel(idxs)
    
ax.errorbar(dllr[idxs], tbl['dv_50'][idxs], 
            yerr=[tbl['dv_15'][idxs], tbl['dv_85'][idxs]], 
            marker='None', ecolor='#aaaaaa', linestyle='none', zorder=-10,
            elinewidth=1.)

fig.tight_layout()
fig.savefig('llr-vs-dv.pdf')

In [ ]:
all_color = '#888888'
comoving_color = '#000000'
alpha = 0.5

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,4.5), sharex=True, sharey=True)

sep2d_bins = sep3d_bins = np.logspace(-2, 1, 13)

_,sep3d_bins,_ = axes[0].hist(tbl['sep_3d'], bins=sep3d_bins, 
                              color=all_color, alpha=alpha, label='all observed')
axes[0].hist(tbl['sep_3d'][tbl['R_RV'] > tbl['R_mu']], bins=sep3d_bins, 
             color=comoving_color, alpha=alpha, 
             label=r'$\mathcal{R}_{\rm RV} > \mathcal{R}_\mu$')
axes[0].set_xlabel('separation [pc]')
axes[0].set_ylabel('$N$')
axes[0].legend(loc='upper left', fontsize=18)

_,sep2d_bins,_ = axes[1].hist(tbl['chord_length'], bins=sep2d_bins, color=all_color, alpha=alpha)
axes[1].hist(tbl['chord_length'][tbl['R_RV'] > tbl['R_mu']], bins=sep2d_bins, color=comoving_color, alpha=alpha)
axes[1].set_xlabel('chord length [pc]')

axes[1].set_xlim(1e-2, 1e1)
axes[1].set_xscale('log')
axes[1].set_yscale('log')

fig.tight_layout()

fig.suptitle('Observed comoving pairs', fontsize=20)
fig.subplots_adjust(top=0.9)

fig.savefig('separation-hist.pdf')

In [ ]:
((tbl['sep_3d'] > 1) & (dllr > 0)).sum() / (tbl['sep_3d'] > 1).sum()

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(6.5,5))

plt.scatter(tbl['sep_3d'], tbl['dv'], marker='.', linewidth=0, alpha=0.75,
            color=all_color, s=15, label='all observed')
plt.scatter(tbl['sep_3d'][tbl['R_RV'] > tbl['R_mu']], tbl['dv'][tbl['R_RV'] > tbl['R_mu']], 
            marker='o', linewidth=0, alpha=0.75, color=comoving_color, s=10,
            label=r'$\mathcal{R}_{\rm RV} > \mathcal{R}_\mu$')
plt.xscale('log')
plt.xlim(1E-2, 1.5E1)
plt.ylim(-5, 100)

ax.set_xlabel(r'$|\boldsymbol{x}_1 - \boldsymbol{x}_2|$ ' + 
              '[{0}]'.format((u.pc).to_string('latex_inline')))
ax.set_ylabel(r'$|\boldsymbol{v}_1 - \boldsymbol{v}_2|$ ' + 
              '[{0}]'.format((u.km/u.s).to_string('latex_inline')))

ax.legend(loc='upper left', fontsize=18)

fig.tight_layout()
fig.savefig('dx-dv.pdf')

## Now plots with RAVE data

In [ ]:
mask = ((all_tbl['R_RV'] > all_tbl['R_mu']) & 
        (all_tbl['sep_3d'].to(u.pc) < 10*u.pc) &
        (all_tbl['d_min'].to(u.pc) < (200.*u.pc)))
print('Total number of confirmed pairs:', mask.sum())

In [ ]:
((rave_tbl['sep_3d'] > 1) & (rave_tbl['R_RV'] > rave_tbl['R_mu'])).sum() / (rave_tbl['sep_3d'] > 1).sum()

In [ ]:
plt.hist(all_tbl['sep_2d'][mask], bins=np.logspace(-3, 1, 15),
         color=comoving_color, alpha=alpha)
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$\Delta \theta$ [deg]')

In [ ]:
sep2d_bins = sep3d_bins = np.logspace(-2, 1, 15)

fig, axes = plt.subplots(1, 2, figsize=(10,4.5), sharex=True, sharey=True)

H,_,_ = axes[0].hist(all_tbl['sep_3d'][mask], bins=sep3d_bins, 
                     color=comoving_color, alpha=alpha)
# _ctr = (sep3d_bins[:-1] + sep3d_bins[1:])/2
# axes[0].errorbar(_ctr, H, np.sqrt(H),
#                  marker='None', linestyle='none', zorder=10, ecolor='k')

axes[0].set_xlabel('separation [pc]')
axes[0].set_ylabel('$N$')

H,_,_ = axes[1].hist(all_tbl['chord_length'][mask], bins=sep2d_bins, 
                     color=comoving_color, alpha=alpha)
# _ctr = (sep2d_bins[:-1] + sep2d_bins[1:])/2
# axes[1].errorbar(_ctr, H, np.sqrt(H),
#                  marker='None', linestyle='none', zorder=10, ecolor='k')

axes[1].set_xlabel('chord length [pc]')

axes[1].set_xlim(1e-2, 1e1)
axes[1].set_xscale('log')
axes[1].set_yscale('log')

fig.tight_layout()

fig.suptitle('Genuine comoving pairs (including RAVE-identified pairs)', 
             fontsize=20)
fig.subplots_adjust(top=0.9)

# fig.savefig('separation-hist-rave.pdf')

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.ticker import LogLocator, AutoMinorLocator

In [ ]:
chord_length = []
dist = []
for shaya_id in np.unique(shaya['Seq']):
    rows = shaya[shaya['Seq'] == shaya_id]
    if len(rows) != 2:
        continue
        
    if rows['Prob'][-1] < 0.5:
        continue
    
    icrs1 = coord.ICRS(ra=rows[0]['_RAJ2000']*u.deg,
                       dec=rows[0]['_DEJ2000']*u.deg)
    icrs2 = coord.ICRS(ra=rows[1]['_RAJ2000']*u.deg,
                       dec=rows[1]['_DEJ2000']*u.deg)
    sep_2d = icrs1.separation(icrs2)
    R = np.mean(rows['Dist'])
    
    dist.append(R)
    chord_length.append((2*R*np.sin(sep_2d/2.)).value)
    
chord_length = u.Quantity(chord_length*u.pc)
dist = u.Quantity(dist*u.pc)

shaya_tbl = Table({'chord_length': chord_length, 'd_min': dist})
len(shaya_tbl)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

divider = make_axes_locatable(ax)
ax_hist_x = divider.append_axes("top", 1.4, pad=0.4, sharex=ax)
ax_hist_y = divider.append_axes("right", 1.4, pad=0.4, sharey=ax)

ax.scatter(tbl['chord_length'], tbl['d_min'],
           marker='o', s=10, color='#67a9cf', label='this work')
ax.scatter(rave_tbl['chord_length'], rave_tbl['d_min'],
           marker='s', s=10, color='#ef8a62', label='RAVE')
ax.set_xscale('log')

xbins = np.logspace(-3, 1, 10)
ybins = np.linspace(0, 200, 10)
ax_hist_x.hist(all_tbl['chord_length'][mask], color=comoving_color, 
               alpha=alpha, bins=xbins)
ax_hist_y.hist(all_tbl['d_min'][mask], bins=ybins, 
               color=comoving_color, alpha=alpha, orientation='horizontal')

ax.legend(loc='lower left', fontsize=16)

ax.set_xlim(2e-3, 1E1)
ax_hist_x.set_xlim(ax.get_xlim())
ax_hist_x.set_yscale('log')
ax_hist_x.set_ylim(3, 2E2)
ax_hist_x.set_yticks([10, 100])
ax_hist_x.yaxis.set_ticks(list(np.arange(2, 10)) + list(np.arange(2, 10)*10), minor=True)
ax_hist_x.set_ylabel('$N$')

ax.set_ylim(0, 200)
ax_hist_y.set_ylim(ax.get_ylim())
ax_hist_y.set_xlim(0, 40)
ax_hist_y.set_xticks([0, 10, 20, 30, 40])
ax_hist_y.set_xlabel('$N$')

# make some labels invisible
plt.setp(ax_hist_x.get_xticklabels() + ax_hist_y.get_yticklabels(),
         visible=False)

ax.set_xlabel(r'chord length, $\hat{s}$ [pc]')
ax.set_ylabel(r'mean distance, $\bar{d}$ [pc]')

fig.savefig('separation-with-rave.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(6, 5))

ax.set_title(r'for $\hat{s} > 0.1\,{\rm pc}$')

ax.hist(all_tbl['chord_length'][mask], color=comoving_color, 
        alpha=alpha, bins=np.linspace(0.1, 10, 7))

ax.set_xlabel(r'chord length, $\hat{s}$ [pc]')
ax.set_ylabel('$N$')

fig.tight_layout()

fig.savefig('linear-separation.pdf')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

divider = make_axes_locatable(ax)
ax_hist_x = divider.append_axes("top", 1.4, pad=0.4, sharex=ax)
ax_hist_y = divider.append_axes("right", 1.4, pad=0.4, sharey=ax)

ax.scatter(tbl['chord_length'], tbl['d_min'],
           marker='o', s=10, color='#67a9cf', label='this work', alpha=0.7)
ax.scatter(rave_tbl['chord_length'], rave_tbl['d_min'],
           marker='s', s=10, color='#ef8a62', label='RAVE', alpha=0.7)
ax.scatter(shaya_tbl['chord_length'], shaya_tbl['d_min'],
           marker='^', s=6, color='#31a354', label='SO11', alpha=0.7)
ax.set_xscale('log')

xbins = np.logspace(-3, 1, 21)
ybins = np.linspace(0, 100, 21)
ax_hist_x.hist(np.concatenate((all_tbl['chord_length'][mask], shaya_tbl['chord_length'])), 
               color=comoving_color, alpha=alpha, bins=xbins)
ax_hist_y.hist(np.concatenate((all_tbl['d_min'][mask], shaya_tbl['d_min'])), bins=ybins, 
               color=comoving_color, alpha=alpha, orientation='horizontal')

ax.legend(loc='lower left', fontsize=12, ncol=3)

ax.set_xlim(2e-3, 1E1)
ax_hist_x.set_xlim(ax.get_xlim())
ax_hist_x.set_yscale('log')
ax_hist_x.set_ylim(8E-1, 1.5E2)
ax_hist_x.set_yticks([1, 10, 100])
ax_hist_x.yaxis.set_ticks(list(np.arange(2, 10)) + list(np.arange(2, 10)*10), minor=True)
ax_hist_x.set_ylabel('$N$')

ax.set_ylim(0, 100)
ax_hist_y.set_ylim(ax.get_ylim())
ax_hist_y.set_xlim(0, 50)
ax_hist_y.set_xticks([0, 50, 100, 150])
ax_hist_y.set_xlabel('$N$')

# make some labels invisible
plt.setp(ax_hist_x.get_xticklabels() + ax_hist_y.get_yticklabels(),
         visible=False)

ax.set_xlabel(r'chord length, $\hat{s}$ [pc]')
ax.set_ylabel(r'mean distance, $\bar{d}$ [pc]')

fig.savefig('separation-with-shaya.pdf')